In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random
import time

class SpectrumEnvironment:
    def __init__(self, num_bands=10):
        self.num_bands = num_bands  # Total number of frequency bands
        self.state = self._generate_spectrum_state()  # Current state of the spectrum
        self.total_reward = 0  # Track total reward
        self.rewards = []  # Track rewards for each step
        self.channel_history = [[] for _ in range(num_bands)]  # Store states for each channel

    def _generate_spectrum_state(self):
        """Randomize the spectrum state: 0 (idle) or 1 (occupied)."""
        return np.random.choice([0, 1], size=self.num_bands)

    def energy_detection(self, band):
        """
        Simulates energy detection for a specific band.
        Higher detection accuracy for idle bands with lower energy.
        """
        noise_level = random.uniform(0, 1)  # Simulate noise
        signal_level = random.uniform(0, 2) if self.state[band] == 1 else random.uniform(0, 0.5)
        detected_energy = signal_level + noise_level
        threshold = 1.0  # Energy threshold for detection
        return detected_energy < threshold

    def matched_filter_fft(self, band):
        """
        Simulates matched filter detection using FFT for specific patterns.
        """
        signal_pattern = np.fft.fft(np.random.randn(10))  # Random pattern
        detected_signal = np.fft.fft(np.random.randn(10) + (self.state[band] * np.random.rand(10)))
        correlation = np.abs(np.dot(signal_pattern.conj(), detected_signal))
        threshold = 15  # Correlation threshold
        return correlation < threshold

    def cyclostationary_detection(self, band):
        """
        Simulates cyclostationary feature detection for periodic properties.
        """
        periodic_property = random.uniform(0, 1) if self.state[band] == 0 else random.uniform(0.5, 1.5)
        threshold = 0.7
        return periodic_property < threshold

    def step(self, action, method = "ED" ):
        """
        Simulate sensing a specific band with advanced detection techniques.
        action: Integer representing the index of the band to sense.
        Returns:
            state: Updated spectrum state.
            reward: Reward based on action.
        """
        reward = 0
        # Combine results from techniques
        if method == "ED":
            hole_detected = self.energy_detection(action)
        elif method == "C":
            hole_detected = self.cyclostationary_detection(action)
        elif method == "MF":
            hole_detected = self.matched_filter_fft(action)
        else:
            return Exception

        # Reward for correctly identifying an idle band
        if hole_detected and self.state[action] == 0:
            reward = 30
        elif not hole_detected and self.state[action] == 1:
            reward = 20
        else:
            reward = -15

        self.total_reward += reward
        self.rewards.append(reward)
        
        # Record the current state in the history
        for i in range(self.num_bands):
            self.channel_history[i].append(self.state[i])
    
        # Randomize the spectrum state for the next step
        self.state = self._generate_spectrum_state()
        return self.state, reward

    def reset(self):
        """Reset the environment to an initial state."""
        self.state = self._generate_spectrum_state()
        self.total_reward = 0
        self.rewards = []
        return self.state
    

